## 四种文档处理链

- stuff
- refine

### **StuffChain**
最常见的文档链，将文档直接塞进Prompt中，为LLM回答问题提供上下文资料，适合小文档的场景
#### **代码示例**

In [11]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama

loader = PyPDFLoader("../03-ChatDoc/files/file.pdf")
print(loader)

prompt_template = """用一句话对以下文字做简要总结：
{text}
总结："""

prompt = PromptTemplate.from_template(prompt_template)

llm = ChatOllama(model="llama3.1:8b")

llm_chain = LLMChain(
  llm=llm,
  prompt=prompt
)

stuff_chain = StuffDocumentsChain(
  llm_chain=llm_chain,
  document_variable_name="text",
  verbose=True
)

docs = loader.load()
result = stuff_chain.run(docs)
print(result)


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)




> Entering new StuffDocumentsChain chain...

> Finished chain.
该合同主要内容包括：合同主体、教学服务内容、教学安排、教学费用的支付方式和退费政策等。


### 使用已经封装好的 load_summarize_chain

In [12]:
from langchain.chains.summarize import load_summarize_chain

load_sum_chain = load_summarize_chain(
  llm=llm,
  chain_type="stuff",
  verbose=True
)

print(load_sum_chain.run(docs))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"电吉他教学服务合同  合同编号：_______________   
甲⽅（学员/家⻓）：   
姓名：___________________________   
身份证号：_______________________   
联系⽅式：_______________________ 
⼄⽅（教师）：   
姓名：___________________________   
身份证号：_______________________   
联系⽅式：_______________________ 
第⼀条 合同主体   
1. 甲⽅为学员本⼈或法定监护⼈，具有签订本合同的权利和能⼒，需按时⽀付教学费⽤并配合教学
安排。   
2. ⼄⽅ _____________________ 专业教师，承诺提供⾼质量的⼀对⼀教学服务。   
第⼆条 教学服务内容 
1. 教学乐器：电吉他（含必要配件，如效果器、拨⽚等）。   
2. 教学⽬标：通过系统教学，帮助学员掌握电吉他基本演奏技巧、⾳阶、和弦、节奏及即兴演奏能
⼒，提升⾳乐表现⼒。   
3. 教学⽅式：采⽤⼀对⼀⾯授⽅式，结合理论讲解、示范演奏与实践练习。   
第三条 教学安排  
1. 服务地点：⼄⽅指定教学场所（详细地址：__________________________________________），
如需变更需提前3⽇协商⼀致。   
2. 单次服务时间：   
   - 每次课程时⻓：_________⼩时（建议45分钟-1⼩时）   
   - 课程周期：⾃_________年_________⽉_________⽇起⾄_________年_________⽉_________⽇
⽌，每周_________次。   
3. 课程阶段：根据学员⽔平动态调整，初期侧重基础技巧，后期逐步增加复杂曲⽬与即兴创作。   
 ⻚码： /1 2

第四条 教学费⽤ 
1. 收费标准：⼈⺠币_________元/课时（

______

## **Refine**
- 通过循环引用LLM， 将文档不断投喂， 并产生各种中间答案，适合逻辑有上下文关联的文档， 不适合交叉引用的文档

### **代码示例**

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain.text_splitter import CharacterTextSplitter

# load
loader = PyPDFLoader("网格算法开发入门指南问答.pdf")
docs = loader.load()

# split
text_split = CharacterTextSplitter.from_tiktoken_encoder(
  chunk_size=1000,
  chunk_overlap=0
)

split_docs = text_split.split_documents(docs)

prompt_template = """用一句话对以下文字做简要总结：
{text}
总结："""

refine_template = (
  "你的任务是产生最终回答\n"
  "我们已经提供了一个到某个特定点的现有回答：{answer}\n"
  "我们有机会通过下面的一些更多上下文来完善现有的回答（仅在需要时使用）。"
  "-------------\n"
  "{text}\n"
  "-------------\n"
  "根据新的上下文，用中文来完善原始回答。"
  "如果上下文没有用处，则返回原始回答。"
)

refine_prompt = PromptTemplate.from_template(refine_template)
llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0)

chain = load_summarize_chain(
  llm=llm,
  chain_type="refine",
  question_prompt=prompt,
  refine_prompt=refine_prompt,
  return_intermediate_steps = True,
  input_key = ["documents","answer"],
  output_key = "output_text",
  verbose=True
)



In [18]:
result = chain({"documents":split_docs, "answer":"网格算法开发入门"}, return_only_outputs=True)
print(result["output_text"])



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
用一句话对以下文字做简要总结：
网格算法开发入门指南 问答
原文网址:https://www.n.cn/search/28e3b68ae5d046d1bdba258c0f1321
44
网格算法开发入门指南（2025 年新版）
一、基础理论构建
1. 数学基础准备
• 计算几何核心
 ▸ 掌握点集拓扑、凸包算法（参考Graham 扫描法）
 ▸ 深入理解Delaunay 三角剖分判定准则（空圆特性与最大化最小角原则）[5]
 ▸ 学习曲面参数化方法（如调和映射、最小二乘保角映射）[3]
• 数值分析基础
 ▸ 网格质量评估指标：雅可比行列式、长宽比、扭曲度计算[6]
 ▸ 误差分析方法：插值误差与h/p 型自适应优化[5]
2. 网格类型认知
网格类型 | 特点 | 典型算法 |
|—————|———————————–|——————————|
 结构化网格| 规则拓扑（四边形/  六面体）| 参数化映射法[3] |
 非结构化网格| 复杂几何适应性好（三角形/  四面体）| Delaunay 剖分[5] |
 混合网格| 边界层棱柱+  核心区四面体| T-Rex 算法[3] |
二、开发技能树搭建
1. 编程能力进阶
• C++深度优化
 ▸ 掌握半边数据结构（Half-Edge）实现[4]
 ▸ 开发高性能网格遍历器（基于R 树或网格索引[1]）
 ▸ 学习VTK 可视化管线开发（实现网格质量着色显示）
以上内容均由AI 搜集并生成，仅供参考
总结：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你的任务是产生最终回答
我们已经提供了一个到某个特定点的现有回答：网格算法开发入门
我们有机会通过下面的一些更多上下文来完善现有的回答（仅在需要时使用）。-------------
• 加速技术实践
 ```cpp
 // CUDA 并行化Delaunay 核心示例
 global void computeDelaunayKernel(Point* 

In [19]:
print("\n\n".join(result["intermediate_steps"][:3]))

<think>
嗯，我现在需要帮用户对一段关于网格算法开发入门指南的中文文章进行简要总结。首先，我得仔细阅读原文，理解里面的主要内容和结构。

原文分为两个主要部分：基础理论构建和开发技能树搭建。在基础理论构建里，作者提到了数学基础准备、数值分析基础以及网格类型认知。数学部分包括计算几何核心，比如点集拓扑、凸包算法、Delaunay三角剖分判定准则、曲面参数化方法等。数值部分涉及网格质量评估指标和误差分析方法。

然后是开发技能树搭建，分为编程能力进阶、数据结构与算法、可视化与交互等方面的内容。编程方面提到了C++深度优化、高性能网格遍历器以及VTK可视化工具。数据结构方面包括半边数据结构和R树或网格索引。可视化部分则涉及网格质量着色显示。

接下来，我需要把这些信息整合成一个简洁的总结，涵盖主要的理论和技能点。要确保每个部分都有简明扼要的概述，并且突出重点。同时，要注意使用用户友好的语言，避免过于专业的术语过多，让读者容易理解。

可能需要注意的是，原文中的某些术语，比如“半边数据结构”、“R树”等，如果读者不是专业人士，可能会觉得有些复杂。所以，在总结时可以适当解释这些术语的含义，或者在必要时用更通俗易懂的语言进行说明。

另外，用户提供的原文中提到了一些具体的算法和工具，如Graham扫描法、Delaunay三角剖分、调和映射等，这些都是关键点，需要在总结中明确提及。同时，数值分析部分提到了雅可比行列式、长宽比和扭曲度的计算，这些也是网格质量评估的重要指标。

最后，我应该确保整个总结逻辑清晰，结构合理，每个主要部分都有简要的概述，并且突出其重要性。这样用户在阅读时能够快速抓住核心内容，而不需要花太多时间去深入研究。
</think>

网格算法开发入门指南（2025 年新版）一、基础理论构建  
1. 数学基础准备  
   - 掌握点集拓扑、凸包算法（参考Graham 扫描法）  
   - 深入理解Delaunay 三角剖分判定准则（空圆特性与最大化最小角原则）[5]  
   - 学习曲面参数化方法（如调和映射、最小二乘保角映射）[3]  
   - 理解网格质量评估指标：雅可比行列式、长宽比、扭曲度计算[6]  

2. 网格类型认知  
   | 核心网格类型 | 特点 | 典型算法 |
   |—————|———————————–|———————